In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Read in the data from Google Drive
trainFilePath = "/content/drive/MyDrive/train.csv"
testFilePath = "/content/drive/MyDrive/test.csv"
train = pd.read_csv(trainFilePath)
test = pd.read_csv(testFilePath)

#Replace NaNs with "None":
train = train.fillna("None")
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,None,Reg,Lvl,AllPub,...,0,None,MnPrv,None,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,None,Reg,Lvl,AllPub,...,0,None,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2010,WD,Normal,142125


In [ ]:
test = test.fillna("None")
test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,None,Reg,Lvl,AllPub,...,120,0,None,MnPrv,None,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,None,IR1,Lvl,AllPub,...,0,0,None,MnPrv,None,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,None,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,None,IR1,HLS,AllPub,...,144,0,None,None,None,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,None,Reg,Lvl,AllPub,...,0,0,None,None,None,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,None,Reg,Lvl,AllPub,...,0,0,None,None,None,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,None,Reg,Lvl,AllPub,...,0,0,None,None,None,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,None,Reg,Lvl,AllPub,...,0,0,None,MnPrv,Shed,700,7,2006,WD,Normal


Random Forest:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
#Drop Sale Price and Id from X and set y to only be Sale Price
X = train.drop(columns = ["Id", "SalePrice"])
y = train.loc[:, train.columns == "SalePrice"]

In [ ]:
#Create the dummy variables
X = pd.get_dummies(X)

#Put y in the format sklearn likes
y = y.to_numpy().flatten().ravel()

In [ ]:
#Get the training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [ ]:
param_grid = {'max_depth': [4, 6, 8, 10, 12],
              'n_estimators': [800, 900, 1000, 1100, 1200, 1300],
              'max_features': [4, 6, 7, 8, 9, 10, 11, 12],
              'bootstrap': [True]}

In [ ]:
random_forest = RandomForestClassifier()

search = GridSearchCV(estimator = random_forest, param_grid = param_grid, cv = 3, n_jobs = -1)

search.fit(X_train, y_train)

search.best_params_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


{'bootstrap': True, 'max_depth': 8, 'max_features': 9, 'n_estimators': 800}

In [ ]:
#Best Hyperparameters: Max Depth = 12, Max Features = 10, N Estimators = 800
y_pred = search.best_estimator_.predict(X_test)

In [ ]:
#RMSE:
np.sqrt(np.mean(np.multiply((y_test - y_pred),(y_test - y_pred))))

62642.87478557066

In [ ]:
X = train.drop(columns = ["Id", "SalePrice"])
y = train.loc[:, train.columns == "SalePrice"]

#This was pulled directly from the Kaggle Competition's Notebook website (Martins, 2023)
reduced_X = X.select_dtypes(include = ['float64', 'int64'])

y = y.to_numpy().flatten().ravel()

X_train, X_test, y_train, y_test = train_test_split(reduced_X, y, test_size = 0.20)

param_grid = {'max_depth': [4, 6, 8, 10, 12],
              'n_estimators': [600, 700, 800, 900, 1000],
              'max_features': [4, 6, 7, 8, 9, 10, 11, 12],
              'bootstrap': [True]}

random_forest = RandomForestClassifier()
search = GridSearchCV(estimator = random_forest, param_grid = param_grid, cv = 3, n_jobs = -1)
search.fit(X_train, y_train)
search.best_params_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'bootstrap': True, 'max_depth': 10, 'max_features': 4, 'n_estimators': 700}

In [ ]:
#Best Hyperparameter Values: Max Depth = 6, Max Features = 9, N Estimators = 800
y_pred = search.best_estimator_.predict(X_test)

#RMSE:
np.sqrt(np.mean(np.multiply((y_test - y_pred),(y_test - y_pred))))

37592.51733363239

In [ ]:
X = train.drop(columns = ["Id", "SalePrice"])
y = train.loc[:, train.columns == "SalePrice"]

#This was pulled directly from the Kaggle Competition's Notebook website (Martins, 2023)
reduced_X = X.select_dtypes(include = ['float64', 'int64'])

y = y.to_numpy().flatten().ravel()

X_train, X_test, y_train, y_test = train_test_split(reduced_X, y, test_size = 0.20)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

#RMSE
np.sqrt(np.mean(np.multiply((y_test - y_pred),(y_test - y_pred))))

35755.63612775872

Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers, layers
import math

In [ ]:
#To normalize the training features data when it gets run through the model
normalize = layers.Normalization()

In [ ]:
N = len(X_train.columns)
m = 1
h1 = math.sqrt((m + 2) * N) + (2* math.sqrt(N / (m + 2)))
h1 = round(h1)
h2 = m * math.sqrt(N / (m + 2))
h2 = round(h2)
print(h1, h2)
#Create the neural network model
model = Sequential([normalize])
model.add(Dense(h1, activation = 'relu'))
model.add(Dense(h2, activation = 'relu'))
model.add(Dense(1)) #1 neuron in output layer

84 17


In [ ]:
model.compile(optimizer = optimizers.Adam(0.001), loss = 'mse')

In [ ]:
#Create the data to be used
X = train.drop(columns = ["Id", "SalePrice"])
y = train.loc[:, train.columns == "SalePrice"]

X = pd.get_dummies(X)

y = y.to_numpy().flatten().ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

#Fit the model
model.fit(X_train, y_train, verbose = 0, epochs = 100)

In [ ]:
#Get the RMSE
mse = model.evaluate(X_test, y_test, verbose=0)
rmse = math.sqrt(mse)
rmse

38701.19874112429

In [ ]:
#Create the neural network model with fewer nodes
model1 = Sequential([normalize])
model1.add(Dense(64, activation = 'relu'))
model1.add(Dense(16, activation = 'relu'))
model1.add(Dense(1)) #1 neuron in output layer

In [ ]:
model1.compile(optimizer = optimizers.Adam(0.001), loss = 'mse')

In [ ]:
#Create the data to be used
X = train.drop(columns = ["Id", "SalePrice"])
y = train.loc[:, train.columns == "SalePrice"]

X = pd.get_dummies(X)

y = y.to_numpy().flatten().ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

#Fit the model
model1.fit(X_train, y_train, verbose = 0, epochs = 100)

In [ ]:
#Get the RMSE
mse = model1.evaluate(X_test, y_test, verbose=0)
rmse = math.sqrt(mse)
rmse

38139.816884720356